In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.1 MB/s eta 0:00:00


In [ ]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.6 MB/s eta 0:00:00


In [ ]:
text =  """BASEL, Switzerland (AP), Roger Federer advanced to the 14th Swiss Indoors final of his career by beating seventh-seeded Daniil Medvedev 6-1, 6-4 on Saturday.
Seeking a ninth title at his hometown event, and a 99th overall, Federer will play 93th-ranked Marius Copil on Sunday.
Federer dominated the 20th-ranked Medvedev and had his first match-point chance to break serve again at 5-1.
He then dropped his serve to love, and let another match point slip in Medvedev's next service game by netting a backhand.
He clinched on his fourth chance when Medvedev netted from the baseline.
Copil upset expectations of a Federer final against Alexander Zverev in a 6-3, 6-7 (6), 6-4 win over the fifth-ranked German in the earlier semifinal.
The Romanian aims for a first title after arriving at Basel without a career win over a top-10 opponent.
Copil has two after also beating No. 6 Marin Cilic in the second round.
Copil fired 26 aces past Zverev and never dropped serve, clinching after 2 1/2 hours with a forehand volley winner to break Zverev for the second time in the semifinal.
He came through two rounds of qualifying last weekend to reach the Basel main draw, including beating Zverev's older brother, Mischa.
Federer had an easier time than in his only previous match against Medvedev, a three-setter at Shanghai two weeks ago."""

# Save the sample text to a .txt file
with open('text.txt', 'w') as file:
    file.write(text)

print("Data has been saved as 'text.txt'")

Data has been saved as 'text.txt'


In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Load clinical reports from a text file
text_reports = []
with open('text.txt', 'r') as file:
    text_reports = [line.strip() for line in file]

# Tokenize the clinical reports
tokenized_reports = [tokenizer.encode(report, add_special_tokens=True) for report in text_reports]

# Pad tokens to the same length
max_length = max(len(tokens) for tokens in tokenized_reports)
padded_reports = [tokens + [tokenizer.pad_token_id] * (max_length - len(tokens)) for tokens in tokenized_reports]

# Convert to PyTorch tensors
input_ids = torch.tensor(padded_reports)

# Create attention masks
attention_masks = torch.ones_like(input_ids)
attention_masks[input_ids == tokenizer.pad_token_id] = 0

# Create a dictionary for mapping tokenized reports to embeddings
report_embeddings = {}
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_masks)
    embeddings = outputs.last_hidden_state
    for i, tokens in enumerate(tokenized_reports):
        report_embeddings[text_reports[i]] = embeddings[i]

In [ ]:
# Calculate sentence importance scores (simplified, using sum of embeddings)
sentence_scores = [embedding.sum().item() for embedding in embeddings]

# Select top sentences based on scores
num_sentences_in_summary = 1
selected_indices = sorted(range(len(sentence_scores)), key=lambda i: sentence_scores[i], reverse=True)[:num_sentences_in_summary]
summary = [text_reports[i] for i in selected_indices]

# Print the summary
print("\n".join(summary))

The Romanian aims for a first title after arriving at Basel without a career win over a top-10 opponent.


In [ ]:
pip install rouge-score

In [ ]:
from rouge_score import rouge_scorer

# Your generated summaries
generated_summaries = [
    ['The Romanian aims for a first title after arriving at Basel without a career win over a top-10 opponent.']
]

# Reference summaries
reference_summaries = [
    ['The Romanian player is targeting her first title in Basel, despite not having won against a top-10 opponent throughout her career.']
]

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])

# Calculate ROUGE scores
total_rouge1 = 0
total_rouge2 = 0
total_rougeL = 0

for gen_summary, ref_summary in zip(generated_summaries, reference_summaries):
    scores = scorer.score(' '.join(gen_summary), ' '.join(ref_summary))
    total_rouge1 += scores['rouge1'].fmeasure
    total_rouge2 += scores['rouge2'].fmeasure
    total_rougeL += scores['rougeL'].fmeasure

average_rouge1 = total_rouge1 / len(generated_summaries)
average_rouge2 = total_rouge2 / len(generated_summaries)
average_rougeL = total_rougeL / len(generated_summaries)

print("Average ROUGE-1 F1:", average_rouge1)
print("Average ROUGE-2 F1:", average_rouge2)
print("Average ROUGE-L F1:", average_rougeL)

Average ROUGE-1 F1: 0.47619047619047616
Average ROUGE-2 F1: 0.25
Average ROUGE-L F1: 0.4285714285714286


In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

import warnings
from torch.optim import AdamW as TorchAdamW



# Load the pre-trained T5 model and tokenizer
model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Read data from the text file
data = []
with open('text.txt', 'r') as file:
    data = [line.strip() for line in file]

# Define a custom dataset class
class SummarizationDataset(Dataset):
    def __init__(self, data, tokenizer, max_source_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_source_length = max_source_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        source_text = self.data[index]
        source_encoding = tokenizer(source_text, truncation=True, padding='max_length',
                                    max_length=self.max_source_length, return_tensors='pt')

        return {
            'source_input_ids': source_encoding.input_ids.squeeze(),
            'source_attention_mask': source_encoding.attention_mask.squeeze()
        }

# Initialize the dataset and data loader
max_source_length = 128
train_dataset = SummarizationDataset(data, tokenizer, max_source_length)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Define optimizer and loss function
warnings.filterwarnings("ignore", category=FutureWarning)
optimizer = TorchAdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        source_input_ids = batch['source_input_ids']

        outputs = model(input_ids=source_input_ids, labels=source_input_ids)

        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Summarization function
def generate_summary(input_text):
    input_encoding = tokenizer(input_text, return_tensors='pt', max_length=max_source_length,
                               truncation=True, padding='max_length')

    summary_ids = model.generate(input_ids=input_encoding.input_ids,
                                 max_length=max_source_length, num_return_sequences=1)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Test the summarization function
input_article = "Major players feel that a big event in late November combined with one in January before the Australian Open will mean too much tennis and too little rest."
summary = generate_summary(input_article)
print("Input Article:", input_article)
print("Generated Summary:", summary)


You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


Input Article: Major players feel that a big event in late November combined with one in January before the Australian Open will mean too much tennis and too little rest.
Generated Summary: players feel that a big event in late November combined with one in January before the Australian Open will mean too much tennis and too little rest.


In [ ]:
#Evaluate the performance of the text summariza-on system using metrics.
!pip install nltk

In [ ]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
pip install rouge

In [ ]:
pip install t5


In [ ]:
pip install sentencepiece

In [ ]:
#Compare the performance of the BERT-based summariza-on system with other state-of the-art summarization syste
from transformers import T5Tokenizer, T5ForConditionalGeneration
from nltk.translate import meteor_score
from nltk.tokenize import word_tokenize
import nltk
from rouge import Rouge

# Download NLTK resources
nltk.download('punkt')

# Load T5 model and tokenizer
model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Example input text
input_text = ("Federer won the Swiss Indoors last week by beating Romanian qualifier Marius Copil in the final. "
              "The stock market experienced a sharp decline today, with tech companies being hit the hardest. "
              "Major players feel that a big event in late November combined with one in January before the Australian Open will mean too much tennis and too little rest.")

# Generate summary using T5 model
input_encoding = tokenizer(input_text, return_tensors='pt', max_length=128, truncation=True, padding='max_length')
output_ids = model.generate(input_ids=input_encoding.input_ids, max_length=32, num_beams=4, length_penalty=2.0, early_stopping=True)
t5_summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Reference summary (ground truth)
reference_summary = "Federer won the Swiss Indoors last week by beating Romanian qualifier Marius Copil in the final."

# Tokenize reference and generated summaries
reference_tokens = word_tokenize(reference_summary)
t5_tokens = word_tokenize(t5_summary)

# Calculate METEOR scores
meteor_t5 = meteor_score.meteor_score([reference_tokens], t5_tokens)

# Print METEOR scores
print("METEOR Score (T5 Abstractive) :", meteor_t5)

# Calculate ROUGE scores
rouge = Rouge()
rouge_scores_t5 = rouge.get_scores(t5_summary, reference_summary)[0]

# Print ROUGE scores
print("ROUGE Scores (T5 Abstractive):", rouge_scores_t5)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


METEOR Score (T5 Abstractive): 0.8157303370786516
ROUGE Scores (T5 Abstractive): {'rouge-1': {'r': 0.8666666666666667, 'p': 0.5909090909090909, 'f': 0.7027026978816656}, 'rouge-2': {'r': 0.8, 'p': 0.5454545454545454, 'f': 0.6486486438276114}, 'rouge-l': {'r': 0.8666666666666667, 'p': 0.5909090909090909, 'f': 0.7027026978816656}}
